# Parameter Identification for Electrochemical Models
## Comparing parameter values identified using reduced-order models

Here we investigate some pitfalls in parameter identification of electrochemical models. We will create synthetic data from the highest-order model in PyBaMM (the many-particle DFN) and then try to estimate the parameter values using the reduced-order models.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import numpy as np
import pybamm

import pybop

go = pybop.plot.PlotlyManager().go
pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

## Generating synthetic data

First, we define a detailed model to generate synthetic data for fitting.

In [ ]:
synth_model = pybamm.lithium_ion.DFN(options={"particle size": "distribution"})
parameter_values = pybamm.ParameterValues("Chen2020")
parameter_values = pybamm.get_size_distribution_parameters(parameter_values)
parameter_values.set_initial_state(0.5)

We can then simulate the model using a custom current function to generate some voltage data.

In [ ]:
n_points = 350
t_eval = np.linspace(0, 1800, n_points)
current = np.concatenate(
    [np.ones(100) * parameter_values["Nominal cell capacity [A.h]"], np.zeros(250)]
)
parameter_values.update(
    {"Current function [A]": pybamm.Interpolant(t_eval, current, pybamm.t)}
)

sim = pybamm.Simulation(synth_model, parameter_values=parameter_values)
sol = sim.solve(t_eval=t_eval)
values = sol["Voltage [V]"](t_eval)

To make the parameter estimation more realistic, we add Gaussian noise to the data.

In [ ]:
sigma = 0.001  # 1 mV
corrupt_values = values + np.random.normal(0, sigma, len(t_eval))
go.Figure(
    data=go.Scatter(x=t_eval, y=corrupt_values, mode="lines"),
    layout=go.Layout(
        title="Corrupted Voltage Data",
        width=800,
        height=600,
        xaxis_title="Time / s",
        yaxis_title="Voltage / V",
    ),
)

We create a dataset for optimisation composed of time, current, and the noisy voltage data.

In [ ]:
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": current,
        "Voltage [V]": corrupt_values,
    }
)

## Identifying the parameters
We will now set up the parameter estimation process by selecting the model parameters we wish to estimate. In this example, we will provide the ground truth values as initial guesses for the parameters - ground truth values are not usually known! But here we are using data generated by a synthetic model as a demonstration.

In [ ]:
true_values = [
    parameter_values["Positive particle diffusivity [m2.s-1]"],
    parameter_values["Negative particle diffusivity [m2.s-1]"],
]
parameters = [
    pybop.Parameter(
        "Positive particle diffusivity [m2.s-1]",
        initial_value=true_values[0],
        bounds=[1e-15, 1e-14],
        transformation=pybop.LogTransformation(),
    ),
    pybop.Parameter(
        "Negative particle diffusivity [m2.s-1]",
        initial_value=true_values[1],
        bounds=[1e-14, 1e-13],
        transformation=pybop.LogTransformation(),
    ),
]

We now define the models that we wish to study.

In [ ]:
models = [
    pybamm.lithium_ion.SPM(),
    pybamm.lithium_ion.SPMe(),
]

For each model, we construct a builder, problem and optimiser.

In [ ]:
optims = []
results = []
for model in models:
    print(f"Optimising the {model.name}")

    builder = (
        pybop.builders.Pybamm()
        .set_dataset(dataset)
        .set_simulation(model, parameter_values=parameter_values)
        .add_cost(pybop.costs.pybamm.SumSquaredError("Voltage [V]", "Voltage [V]"))
    )
    for param in parameters:
        builder.add_parameter(param)
    problem = builder.build()

    options = pybop.PintsOptions(
        max_unchanged_iterations=10,
        max_iterations=60,
    )
    optim = pybop.XNES(problem, options=options)
    res = optim.run()

    optims.append(optim)
    results.append(res)

Optimising the Single Particle Model


Optimising the Single Particle Model with electrolyte


In [ ]:
print(f"| Synthetic Model | True values: {true_values} |")
for model, res in zip(models, results, strict=False):
    print(f"| Model: {model.name} | Results: {res.x} |")

| Synthetic Model | True values: [4e-15, 3.3e-14] |
| Model: Single Particle Model | Results: [1.78150052e-15 1.18754960e-14] |
| Model: Single Particle Model with electrolyte | Results: [3.21383388e-15 9.97618998e-14] |


## Plotting and Visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation. First let's look at the voltage response of each model compared to the data.

In [ ]:
for model, res in zip(models, results, strict=False):
    sim = pybamm.Simulation(model, parameter_values=res.parameter_values)
    t_eval = dataset["Time [s]"]
    sol = sim.solve(t_eval=t_eval)
    pybop.plot.trajectories(
        x=t_eval,
        y=[dataset["Voltage [V]"], sol["Voltage [V]"](t_eval)],
        trace_names=["Dataset", "Simulation"],
        xaxis_title="Time / s",
        yaxis_title="Voltage / V",
    )

### Cost Landscape

We can visualise the cost landscape and the path taken by the optimiser:


In [ ]:
for model, optim in zip(models, optims, strict=False):
    pybop.plot.surface(optim, title=model.name)

## Concluding thoughts

In this example, the optimised values appear to differ from the ground truth values of the effective diffusivity parameters. Why is this?

There are two factors at play in this example. Firstly, it is important to remember that parameters are only defined with respect to a specific model, despite sharing the same name. The DFN, SPMe and SPM vary in their description of diffusion dynamics. Therefore different values can be identified for nominally the same parameter when identification is performed using models of differing complexity. In the case of the SPM, it is not possible to obtain a close fit and the optimised values are a compromise.

The SPMe achieves a better fit of the data, but only the "Positive particle diffusivity [m2.s-1]" can be identified. The "Negative particle diffusivity [m2.s-1]" has poor identifiablity in this case, since this parameter has a negligible impact on the voltage response of this cell under these conditions.